**Resumes**

Sorry for downloading your resume. I need it for testing

In [3]:
resume = "./resumes/cw-resume-jv1.pdf"

In [ ]:
resume = "./resumes/Jerick-Iquin.pdf"

In [5]:
resume = "./resumes/Kenneth Lee Resume.pdf"

In [ ]:
resume = "./resumes/Justin Lam - Resume - vPortfolio.pdf"

**Using pymupdf**

- Best one so far. Plus you can organize it through blocks

In [15]:
import fitz # this is from pymupdf
from unidecode import unidecode
import pandas as pd
import pprint

doc = fitz.open(resume)
block_dict = {}

page_num = 0

for page in doc:
    page_num += 1
    file_dict = page.get_text('dict')
    # block format (x0, y0, x1, y1, string, block_no, block_type)
    block = file_dict['blocks']
    block_dict[page_num] = block    

In [19]:
import re 

spans = pd.DataFrame(columns=['xmin', 'ymin', 'xmax', 'ymax', 'text', 'tag'])
rows = []

for page_num, blocks in block_dict.items():
    for block in blocks:
        if block['type'] == 0:
            for line in block['lines']:
                for span in line['spans']:
                    xmin, ymin, xmax, ymax = list(span['bbox'])
                    font_size = span['size']
                    text = unidecode(span['text'])
                    span_font = span['font']
                    is_upper = False
                    is_bold = False 

                    if "bold" in span_font.lower():
                        is_bold = True 

                    if re.sub("[\(\[].*?[\)\]]", "", text).isupper():
                        is_upper = True

                    if text.replace(" ","") !=  "":
                        rows.append((xmin, ymin, xmax, ymax, text, is_upper, is_bold, span_font, font_size))

span_df = pd.DataFrame(rows, columns=['xmin','ymin','xmax','ymax', 'text', 'is_upper','is_bold','span_font', 'font_size'])

In [26]:
import numpy as np

span_scores = []
span_num_occur = {}
special = '[(_:/,#%\=@)]'

for index, span_row in span_df.iterrows():
    score = round(span_row.font_size)
    text = span_row.text

    if not re.search(special, text):
        if span_row.is_bold:
            score += 1

        if span_row.is_upper:
            score += 1
    
    span_scores.append(score)

values, counts = np.unique(span_scores, return_counts=True)

style_dict = {}

for value, count in zip(values, counts):
    style_dict[value] = count

sorted(style_dict.items(), key=lambda x: x[1])

p_size = max(style_dict, key=style_dict.get)

idx = 0
tag = {}

for size in sorted(values, reverse = True):
    idx += 1

    if size == p_size:
        idx = 0
        tag[size] = 'p'

    if size > p_size:
        tag[size] = 'h{0}'.format(idx)
    
    if size < p_size:
        tag[size] = 's{0}'.format(idx)

span_tags = [tag[score] for score in span_scores]
span_df['tag'] = span_tags

**Using pdfminer.six**

- Second best. Not as good as pymupdf

In [ ]:
from pdfminer.high_level import extract_text, extract_pages
from pdfminer.layout import LAParams

laparams = LAParams(line_margin=1, boxes_flow=1)

In [ ]:
# extract pages outputs an iterable so we can create list from it
pages = [x for x in extract_pages(resume)]

for page in range(len(pages)):
    print('page_number', page)
    text = extract_text(resume, page_numbers=page, laparams=laparams)
    print(text)

**Testing Beautiful Soup**

This is to do webscraping of your linkedin profile

- Linkedin detects automated scraping and blocks requests
- Need to install chrome and chromedriver
- Issue with linkedin possibly blocking webscraping
- Will have different classnames when actually loggedin
- Possibly go for linkedin api?

In [ ]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from bs4 import BeautifulSoup
import time

def scrape_linkedin_profile(url):
    options = Options()
    # running in headless mode
    options.add_argument('--headless') 
    driver = webdriver.Chrome(executable_path='chromedriver', options=options)

    driver.get(url)

    # wait for the page to load. adjust sleep here
    # driver.implicitly_wait(10)
    time.sleep(5)

    html_content = driver.page_source

    soup = BeautifulSoup(html_content, 'html.parser')
    print(soup)

    lis = soup.find_all('li', class_='experience-item')

    try:
        modal_dismis = driver.find_element(By.CLASS_NAME, 'contextual-sign-in-modal__modal-dismiss')
        if modal_dismis:
            modal_dismis.click()
    except Exception:
        print('Already opened')


    show_more_buttons = driver.find_elements(By.CLASS_NAME, 'show-more-less-text__button--more')

    for button in show_more_buttons:
        button.click()

    for li in lis:
        print("Experience", lis.index(li))
        description_more = ''
        description_less = ''
        
        title = get_text(li.find('h3', class_='profile-section-card__title'))
        company = get_text(li.find('h4', class_='profile-section-card__subtitle'))
        location = get_text(li.find('p', class_='experience-item__location'))

        # the class naming is weird in this case. Everything that has show more button will have --more
        # modifier and everything else has --less modifier 
        description_more = get_text(li.find('p', class_='show-more-less-text__text--more'))
        description_less = get_text(li.find('p', class_='show-more-less-text__text--less'))

        if description_more:
            description_list = get_duties(description_more)
        elif description_less:
            description_list = get_duties(description_less)
        else:
            description_list = []
            
        print(title)
        print(company)
        print(location)
        print('Responsibilities: ')
        for duty in description_list:
            print(duty)
        print('\n')
        
    driver.quit()

def get_text(element):
    try:
        return element.get_text().strip()
    except Exception as e:
        return ''
    
def get_duties(description):
    # for some reason the button "show less" keeps popping up. Need to remove that
    trimmed = description.replace('\n            \n\n    \n    \n\n    \n        Show less', '')
    return trimmed.split('• ')[1:]


scrape_linkedin_profile('https://www.linkedin.com/in/jerick-iquin/')

**Using pypdf package**

- Didn't work on Jek's email. Some of the spaces are not being extracted.

In [ ]:
from pypdf import PdfReader

reader = PdfReader(resume)
number_of_pages = len(reader.pages)

for page_number in range(number_of_pages):
    page = reader.pages[page_number]
    text = page.extract_text()
    print(text)

**Using pyPDF2**

- Didn't work on Jek's email. Some of the spaces are not being extracted.

In [ ]:
from PyPDF2 import PdfReader

reader = PdfReader(resume, strict=True)
number_of_pages = len(reader.pages)
for page_number in range(number_of_pages):
    page = reader.pages[page_number]
    text = page.extract_text()
    print(text)
